In [9]:
pip uninstall transformers -y


Found existing installation: transformers 4.39.3
Uninstalling transformers-4.39.3:
  Successfully uninstalled transformers-4.39.3
Note: you may need to restart the kernel to use updated packages.


In [4]:
!rm -rf /home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers
!rm -rf ~/.cache/huggingface/transformers
!rm -rf ~/.cache/torch/transformers
!rm -rf ~/.cache/huggingface/modules/transformers_modules
!find /home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/ -name "*transformers*" -exec rm -rf {} +


In [2]:
pip install transformers==4.39.3 peft==0.7.1 datasets accelerate


  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
Using cached transformers-4.39.3-py3-none-any.whl (8.8 MB)
Note: you may need to restart the kernel to use updated packages.


In [3]:
!pip uninstall pandas datasets -y
!pip install pandas==2.1.4 datasets==3.6.0


Found existing installation: pandas 2.1.4
Uninstalling pandas-2.1.4:
  Successfully uninstalled pandas-2.1.4
Found existing installation: datasets 3.6.0
Uninstalling datasets-3.6.0:
  Successfully uninstalled datasets-3.6.0
  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
  Using cached fsspec-2025.3.0-py3-none-any.whl.metadata (11 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 161.3 MB/s eta 0:00:00
Using cached datasets-3.6.0-py3-none-any.whl (491 kB)
Using cached fsspec-2025.3.0-py3-none-any.whl (193 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 174.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.2.6
    Uninstalling numpy-2.2.6:
      Successfully uninstalled numpy-2.2.6
  Attempting uninstall: fsspec━━━━━━━━━━━━━━━━━━ 0/4 [numpy]
    Found existing installation: fsspec 2025.5.1 0/4 [numpy]
    Uninstalling fsspec-2025.5.1:━━━━━━━━━━━ 0/4 [numpy]
      Successfully uninstalled fsspec-2025.5.10m 0/

In [3]:
pip install accelerate==0.27.2 --force-reinstall


  Using cached huggingface_hub-0.33.0-py3-none-any.whl.metadata (14 kB)
  Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (3.8 kB)
  Using cached hf_xet-1.1.3-cp37-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (879 bytes)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.8/16.8 MB 195.7 MB/s eta 0:00:00
Using cached safetensors-0.5.3-cp38-abi3-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (471 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 821.2/821.2 MB 115.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 177.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 190.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 202.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 116.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 147.0 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━

In [1]:
import transformers
print(transformers.__version__)
print(transformers.__file__)


4.39.3
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/__init__.py


In [ ]:
# 1. Imports
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainer,
    Seq2SeqTrainingArguments,
    DataCollatorForSeq2Seq
)
from peft import get_peft_model, LoraConfig, TaskType
import torch

# 2. Load CNN/DailyMail dataset
dataset = load_dataset("cnn_dailymail", "3.0.0")
train_data = dataset["train"]
val_data = dataset["validation"]

# 3. Load tokenizer and base model
model_name = "t5-small"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# 4. Apply LoRA configuration
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q", "v"],      
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.SEQ_2_SEQ_LM
)
model = get_peft_model(model, peft_config)

# 5. Preprocessing function
def preprocess(batch):
    inputs = ["summarize: " + article for article in batch["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(batch["highlights"], max_length=128, truncation=True, padding="max_length")
    model_inputs["labels"] = labels["input_ids"]

    return model_inputs

# 6. Tokenize dataset
tokenized_train = train_data.map(preprocess, batched=True, remove_columns=train_data.column_names, num_proc=4)
tokenized_val = val_data.map(preprocess, batched=True, remove_columns=val_data.column_names, num_proc=4)

# 7. Set training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./full_lora_summarizer",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-4,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    weight_decay=0.01,
    save_total_limit=2,
    num_train_epochs=3,
    predict_with_generate=True,
    logging_dir="./logs",
    fp16=torch.cuda.is_available(),
    report_to="none"
)

# 8. Define data collator
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

# 9. Initialize trainer



/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
    tokenizer=tokenizer,
    data_collator=data_collator
)

# 10. Train model
trainer.train()

# 11. Save model & tokenizer
model.save_pretrained("./full_lora_summarizer")
tokenizer.save_pretrained("./full_lora_summarizer") 

print("✅ LoRA fine-tuning complete! Model saved at ./full_lora_summarizer")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/accelerate/accelerator.py:450: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.scaler = torch.cuda.amp.GradScaler(**kwargs)


Epoch,Training Loss,Validation Loss
1,1.164600,1.154472
2,1.167000,1.149502
3,1.147400,1.147624


✅ LoRA fine-tuning complete! Model saved at ./full_lora_summarizer


In [1]:
pip install gradio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.3/54.3 MB 148.4 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 188.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13/13 [gradio]12/13 [gradio]
Note: you may need to restart the kernel to use updated packages.


In [10]:
# 14. Gradio UI for Live Summarization
import gradio as gr
import torch  # ← Missing import

# Load model and tokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer

model_path = "./full_lora_summarizer"
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


def summarize(text):
    input_ids = tokenizer("summarize: " + text, return_tensors="pt", truncation=True, padding=True, max_length=2048).input_ids
    output_idse = model.generate(
    input_ids,
    do_sample=True,
   
    top_p=0.95,
    max_length=500,
)
    with torch.no_grad():
        output_ids = output_idse
    summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return summary

# Gradio interface
interface = gr.Interface(
    fn=summarize,
    inputs=gr.Textbox(lines=10, label="Enter Article/News/Content"),
    outputs=gr.Textbox(lines=5, label="Summary"),
    title="📝 Summarization with LoRA + T5",
    description="Powered by HuggingFace Transformers + PEFT (LoRA). Input a long text and get the summarized output instantly."
)

interface.launch(share=True)  # share=True to get a public URL


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://3c2bc85b148f01b429.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [2]:
pip install transformers datasets evaluate


Note: you may need to restart the kernel to use updated packages.


In [4]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments
from datasets import load_dataset
import torch

# Load your saved model
model_path = "./T5-text-summarizer/full_lora_summarizer"  # Change if your path is different
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)


/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/242M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [5]:
dataset = load_dataset("cnn_dailymail", "3.0.0", split="validation[:100]")  # You can use [:1000] for more samples


README.md:   0%|          | 0.00/15.6k [00:00<?, ?B/s]

train-00000-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00001-of-00003.parquet:   0%|          | 0.00/257M [00:00<?, ?B/s]

train-00002-of-00003.parquet:   0%|          | 0.00/259M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/34.7M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/30.0M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/287113 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/13368 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/11490 [00:00<?, ? examples/s]

In [6]:
def preprocess_function(examples):
    inputs = ["summarize: " + doc for doc in examples["article"]]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")

    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples["highlights"], max_length=128, truncation=True, padding="max_length")

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_val = dataset.map(preprocess_function, batched=True, remove_columns=dataset.column_names)


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [8]:
!pip install nltk rouge_score


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 56.1 MB/s eta 0:00:00
  DEPRECATION: Building 'rouge_score' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'rouge_score'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24987 sha256=0e6e0c14764b6bfd71c2b9a7fbecfb198b194214c0dfbc340fe3c89f0f2cd656
  Stored in directory: /home/zeus/.cache/pip/wheels/5f/dd/89/461065a73be61a532ff8599a28e9beef17985c9e9c31e541b4
Successfully built rouge_score
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [rouge_score]


In [10]:
from evaluate import load
import numpy as np

# Load ROUGE metric
rouge = load("rouge")

# Get model predictions
predictions = trainer.predict(tokenized_val)

# Convert logits to token IDs before decoding
pred_ids = np.argmax(predictions.predictions, axis=-1)
decoded_preds = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
decoded_labels = tokenizer.batch_decode(predictions.label_ids, skip_special_tokens=True)

# Compute ROUGE scores
results = rouge.compute(predictions=decoded_preds, references=decoded_labels)
print("📊 ROUGE Evaluation Results:")
for k, v in results.items():
    print(f"{k}: {v:.4f}")


📊 ROUGE Evaluation Results:
rouge1: 0.0036
rouge2: 0.0000
rougeL: 0.0036
rougeLsum: 0.0036
